1) Imoprt Dependencies

pip install stable-baselines3[extra]

https://stable-baselines3.readthedocs.io/en/master/

In [1]:
import os                                                           # operating system library
import gymnasium as gym                                             # for openAI gymnasium (which has replaced openai gym)
from stable_baselines3 import PPO                                   # algorithm
# Action Spaces: Algorithms
#   Discrete Single Process: DQN
#   Discrete Multi Professed: PPO or A2C
#   Continuous Single Process: SAC or TD3
#   Continuous Multi Processed: PPO or A2C

from stable_baselines3.common.vec_env import DummyVecEnv            # vectorizing environment allows train on multiple environments at the same time (this doesn't do that)
from stable_baselines3.common.evaluation import evaluate_policy     # mean and stdev of reward

2) Load Environment

In [2]:
# Create environment name
environment_name = 'CartPole-v1'

In [2]:
# create the environment
env = gym.make(environment_name,render_mode = 'human')

print(environment_name)


CartPole-v1


Understanding the Environment

In [3]:
# two different spaces:
# action space (actions we can do)
print(env.action_space)
print(env.action_space.sample())

# and observations (things we observe)
print(env.observation_space)
print(env.observation_space.sample())

state = env.reset()
print(env.step(1))

Discrete(2)
1
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
[ 3.30346847e+00 -5.53655780e+36  1.14650585e-01  2.20745309e+38]


In [8]:
# loop for testing environment
episodes = 5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        # make the render
        env.render()

        # select the next action
        action = env.action_space.sample()

        # take a step using the action and return the new state, reward, is the episode done? x2, info
        n_state, reward, terminated, truncated, info = env.step(action)
        done = truncated or terminated

        

        # aggregate the reward
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:17.0
Episode:2 Score:13.0
Episode:3 Score:28.0
Episode:4 Score:61.0
Episode:5 Score:33.0


3) Train an RL Model

In [8]:
# where we save our tensorboard log
log_path = os.path.join('Training','Logs')

# recreate the environment
environment_name = 'CartPole-v1'
env = gym.make(environment_name)

# wrap environ in dummy vec environment (wrapper for non-vectorized environment)
env = DummyVecEnv([lambda: env])

# create the model: multilayer neural network policy (the rules teh agent uses to determine its actions)
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

# train model (including number of timesteps to train model for)
model.learn(total_timesteps=20000)

Using cuda device
Logging to Training\Logs\PPO_16
-----------------------------
| time/              |      |
|    fps             | 975  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 733        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00886639 |
|    clip_fraction        | 0.0907     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | -0.00117   |
|    learning_rate        | 0.0003     |
|    loss                 | 7.76       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.013     |
|    value_loss           | 49.9       |
----------------------------------------
--

4) Save and Reload Model

In [4]:
# path to save model to
PPO_Path = os.path.join('Training','Saved Models','PPO_Model_Cartpole')

In [9]:
# test saving an deleting the model
model.save(PPO_Path)
del model
model = PPO.load(PPO_Path,env=env)

5) Evaluation

In [11]:
# score of 200 or higher is generally considered solved for PPO

# load model
env = gym.make(environment_name,render_mode = 'human')
model = PPO.load(PPO_Path,env=env)

# returns mean and std of rewards
evaluate_policy(model,env,n_eval_episodes=10,render=True)
env.close()

(465.6, 47.028076720189176)

: 

6) Test Model

In [5]:
# take observations, pass them to agent, agent will determine action using learned policy

# load model
env = gym.make(environment_name,render_mode = 'human')
model = PPO.load(PPO_Path,env=env)

# loop for testing environment
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        # make the render
        env.render()

        # select the next action using our model!!
        action = model.predict()

        # take a step using the action and return the new state, reward, is the episode done? x2, info
        obs, reward, terminated, truncated, info = env.step(action)
        done = truncated or terminated

        # aggregate the reward
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))
env.close()

NameError: name 'env' is not defined